In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import os

In [2]:
os.getcwd()

'/Users/oraff/Downloads'

In [3]:
df = pd.read_csv('AfterEDAChiCrime.csv')

In [4]:
df['Date'] = pd.to_datetime(df['Date'])

df.head()

,Unnamed: 0,ID,Date,Primary Type,Arrest,Domestic,District Name,Average Daily Wind Speed Mph,Fastest mile Wind Speed time,Peak gust time,...,Fog/Ice Fog,Heavy Fog,Thunder,Hail,Smoke/Haze,Tornado,Year,Month,Day_name,Severity
0,0,11109587,2001-01-01,DECEPTIVE PRACTICE,0,0,Rogers Park,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
1,1,1780042,2001-01-01,OTHER OFFENSE,0,0,Morgan Park,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
2,2,5691515,2001-01-01,THEFT,0,0,Englewood,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0
3,3,5233406,2001-01-01,THEFT,1,0,Near West,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,1
4,4,1310513,2001-01-01,THEFT,0,0,Ogden,7.38,1519.0,1518.0,...,0,0,0,0,0,0,2001,January,Monday,0


In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df1 = df['Primary Type'].str.get_dummies()

In [ ]:
df = pd.concat([df, df1], axis=1)

df.head()

In [ ]:
df2 = df['District Name'].str.get_dummies()

In [ ]:
df = pd.concat([df, df2], axis=1)

df.head()

In [ ]:
df3 = df['Day_name'].str.get_dummies()

df = pd.concat([df, df3], axis=1)

df.head()

In [ ]:
df4 = df['Month'].str.get_dummies()

df = pd.concat([df, df4], axis=1)

df.head()

In [ ]:
df['Hour'] = df.Date.dt.hour
df['Day_of_year'] = df.Date.dt.dayofyear
df['Quarter'] = df.Date.dt.quarter

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=df['Date'].min(), end=df['Date'].max())

df['Holiday'] = df['Date'].isin(holidays)


In [ ]:
df = df.drop(columns=['ID', 'Primary Type', 'District Name', 'Month', 'Day_name', 'Date'], axis=1)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
df_severe = df[df.Severity != 0]
df_non = df[df.Severity != 1]
df_severe_70 = df_severe.sample(n=70000)
df_non_70 = df_non.sample(n=70000)
df_tnt = df_severe_70.append(df_non_70)

In [ ]:
X = df_tnt.drop(columns=['Severity', 'Arrest'], axis=1)
y = df_tnt['Severity']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(X_train, y_train)

In [ ]:
features = list(IV[(IV['IV'] >= 0.001) & (IV['IV'] <= 0.8)]['VAR_NAME'])
X2 = X_train[features]

In [ ]:
def iterate_vif(df, vif_threshold=5, max_vif=6):
  count = 0
  while max_vif > vif_threshold:
    count += 1
    print("Iteration # "+str(count))
    vif = pd.DataFrame()
    vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns
    
    if vif['VIFactor'].max() > vif_threshold:
      print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
      df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
      max_vif = vif['VIFactor'].max()
    else:
        print('Complete')
        return df, vif.sort_values('VIFactor')
 
final_df, final_vif = iterate_vif(X2)

In [ ]:
X_train=final_df

In [ ]:
X_test=X_test[X_train.columns]

In [ ]:
#XGBcalssifier Model 
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100,learning_rate=.1, max_depth=10, seed=1)

In [ ]:
xg_cl.fit(X_train, y_train)

In [ ]:
y_pred = xg_cl.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test, y_pred)
conf

In [ ]:
import shap
shap_values = shap.TreeExplainer(xg_cl).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100, max_depth=10, random_state=5, )
clf.fit(X_train,y_train)

In [ ]:
y_pred1 = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred1)

In [ ]:
conf = confusion_matrix(y_test, y_pred1)
conf

In [ ]:
shap_values = shap.TreeExplainer(clf).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
#LightGbm here
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(max_depth=10, n_estimators=100, random_state=5)
lgb_clf.fit(X_train, y_train)

In [ ]:
y_pred2 = lbg_clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred2)

In [ ]:
conf = confusion_matrix(y_test, y_pred2)
conf

In [ ]:
shap_values = shap.TreeExplainer(lgb_clf).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")